In [6]:
import os
import dotenv
dotenv.load_dotenv()
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API")

model = ChatGroq(model = "openai/gpt-oss-120b", api_key = groq_api_key)


In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

### Prompt Templates

Prompt templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, whihc are we passing to the LLM.

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model
chain.invoke({"messages":[HumanMessage(content="hi my name is Pourush gupta and I am one of the greates AI engineer")]})

AIMessage(content='Hello Pourush! 👋 It’s great to meet an AI engineer—especially one who’s proud of their work. How can I assist you today? Whether you’ve got a technical question, need help brainstorming a project, or just want to chat about the latest trends in AI, I’m here to help!', additional_kwargs={'reasoning_content': 'We need to respond as ChatGPT. The user says "hi my name is Pourush gupta and I am one of the greatest AI engineer". Likely they are introducing themselves. We can respond politely, ask how we can help, maybe compliment. Should be friendly.'}, response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 107, 'total_tokens': 235, 'completion_time': 0.29659896, 'completion_tokens_details': {'reasoning_tokens': 56}, 'prompt_time': 0.004245186, 'prompt_tokens_details': None, 'queue_time': 0.047835224, 'total_time': 0.300844146}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_1d1727abc9', 'service_tier': 'on_demand', 'finish_reason': '

In [13]:
config = {"configurable":{"session_id":"chat1"}}

response = with_message_history.invoke(
    [HumanMessage(content="hi this side Pourush Gupta, one of the greates AI engineer")],
    config=config
)

print(response.content)

Hello, Pourush! Great to meet you. How can I assist you today?


In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer the user query in {language} language"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

response = chain.invoke({"messages":[HumanMessage(content="hi my name is Pourush Gupta and I am one of the greatest AI engineer")], "language":"hindi"})

print(response.content)


नमस्ते पूरुष गुप्ता! आपका स्वागत है। यह सुनकर बहुत खुशी हुई कि आप एक महान AI इंजीनियर हैं। यदि आपको किसी प्रोजेक्ट, तकनीकी सवाल, या किसी भी तरह की मदद चाहिए, तो बेझिझक बताइए—मैं यहाँ मदद के लिए हूँ! 🚀


In [18]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
config = {"configurable":{"session_id":"chat2"}}

response = with_message_history.invoke(
    {"messages":[HumanMessage(content="hi my name is Pourush Gupta and I am of the greates AI engineer")],"language":"hindi"},
    config=config
)

print(response.content)


नमस्ते पूरुष गुप्ता! आपका स्वागत है। आप एक महान AI इंजीनियर हैं—यह सुनकर बहुत खुशी हुई। मैं आपकी कैसे मदद कर सकता हूँ? यदि कोई सवाल है या किसी प्रोजेक्ट में सहायता चाहिए, तो बेझिझक बताइए।
